## 7.1 了解RNN的工作原理

In [1]:
# 循环神经网络(Recurrent neural network,RNN)是一类用于处理序列数据的神经网络
# 类比前馈神经网络，当前馈神经网络被扩展成在模型输出与模型输入之间包含反馈连接时，它们被称为循环神经网络

# 在模型的不同部分共享参数。参数共享使得模型能够扩展到不同形式的样本(这里指不同形式的样本)并进行泛化
# 如果在每个时间点都有一个单独的参数
# 不但不能泛化到训练时没有见过的长度，也不能在时间上共享不同序列长度和不同位置的统计强度
# 当信息的特定部分会在序列内多个位置出现时，这样的共享尤其重要

# RNN可以处理扩展到更长的序列，大多数RNN也可以处理可变长度序列
# RNN网路使用随时间反向传播(Backpropagation Through Time, BPTT)的链式求导算法来反向传播误差

# 7.2 简单RNN

### 7.2.1 简单循环神经网络的实现——裸写一个退位减法器

In [32]:
#规定随机数生成器的种子，可以每次得到一样的值
np.random.seed(0)
'''
一 定义基本函数
'''

def sigmoid(x):
    '''
    定义S型函数
    
    args:
        x：输入数或list、ndarray
    '''
    return 1/(1+np.exp(-x))
    
def sigmoid_output_to_derivative(output):
    '''
    定义sigmod的导数
    
    args:
        output:sigmoid函数的输出  假设要计算x=5时，sigmoid函数的导数，此处就传入sigmoid(5)
    '''
    return output*(1-output)
'''
二 建立二进制映射

定义的减法最大值限制在256以内，即8位二进制的减法，定义int与二进制之间的映射字典int2binary
'''
#整数到其二进制表示的映射字典
int2binary = {}
#二进制的位数
binary_dim = 8
#计算0-255的二进制表示
largest_number = pow(2,binary_dim)
'''
注意 np.array([range(largest_number)],dtype=np.uint8) 返回的是[[0,1,2,3...255]] 形状1x256 如果使用这个后面需要.T进行转置
np.array(range(largest_number),dtype=np.uint8) 返回的是[0,1,2,...255]形状为(256,) 尽量不使用这种形状不明确的
然后按行转为二进制 得到256x8
'''
binary = np.unpackbits(
        np.array(range(largest_number),dtype=np.uint8).reshape(256,1),axis=1)
#建立int-二进制映射
for i in range(largest_number):
    #向字典中追加数据
    int2binary[i] = binary[i]
'''
三 定义参数

隐藏层的权重synapse_0(2x16)，输出层的权重synapse_1(16x1)，循环节点的权重synapse_h(16x16)
这里只设置权重 忽略偏置
'''

#参数设置
learning_rate = 0.9                 #学习速率
input_dim = 2                       #输入节点的个数为2，减数和被减数
hidden_dim = 16                     #隐藏层节点个数
output_dim = 1                      #输出节点个数

n_samples = 10000                   #样本个数 

#初始化网络  np.random.random生成一个[0,1)之间随机浮点数或size大小浮点数组
synapse_0 = (2*np.random.random((input_dim,hidden_dim))-1)*0.05   #-0.05~0.05之间
synapse_1 = (2*np.random.random((hidden_dim,output_dim))-1)*0.05  #-0.05~0.05之间
synapse_h = (2*np.random.random((hidden_dim,hidden_dim))-1)*0.05   #-0.05~0.05之间

#用于存放反向传播的权重梯度值
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)
'''
四 准备样本数据
'''


#建立循环生成样本数据，先生成两个数a,b，如果a小于b，就交换位置，保证被减数大
for i in range(n_samples):
    #生成一个数字a 被减数  范围[0,256)之间的整数
    a_int = np.random.randint(largest_number)   
    #生成一个数字b 减数，b的最大值取得是largest_number/2 
    b_int = np.random.randint(largest_number/2)   
    #如果生成的b>a交换
    if a_int < b_int:
        tmp = b_int
        b_int = a_int
        a_int = tmp
     
    #二进制编码
    a = int2binary[a_int]    #被减数
    b = int2binary[b_int]    #减数
    c = int2binary[a_int - b_int]  #差值
    '''
    五 模型初始化
    '''
    d = np.zeros_like(c)     #存储神经网络的预测值 初始化为0
    over_all_error = 0       #初始化总误差为0
    
    layer_2_deltas = list()  #存储每个时间点输出层的误差
    layer_1_values = list()  #存储每个时间点隐藏层的值
    
    layer_1_values.append(np.ones(hidden_dim)*0.1)  #一开始没有隐藏层(t=1)，所以初始化原始值为0.1
    '''
    六 正向传播
    '''
    #循环遍历每一个二进制位
    for position in range(binary_dim):
        #生成输入和输出 从右向左，每次取两个输入数字的一个bit位
        X = np.array([[a[binary_dim - position- 1],b[binary_dim - position - 1]]])
        #正确答案
        y = np.array([[c[binary_dim - position -1]]]).T
        
        #计算隐藏层输出 新的隐藏层 = 输入层 + 之前的隐藏层
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))        
        #将隐藏层保存下来，下个事件序列可以使用
        layer_1_values.append(copy.deepcopy(layer_1))
        
        #计算输出层
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))
        
        #预测误差
        layer_2_error = layer_2 - y
        #把每个时间点的误差导数都记录下来
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        
        #总误差
        over_all_error += np.abs(layer_2_error[0])
        
        #记录每个预测的bit位
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
    
    fuature_layer_1_delta = np.zeros(hidden_dim)
    '''
    七 反向训练
    '''
    for position in range(binary_dim):
        X = np.array([[a[position],b[position]]])     #最后一次的两个输入
        layer_1 = layer_1_values[-position-1]         #当前时间点的隐藏层
        prev_layer_1 = layer_1_values[-position-2]    #前一个时间点的隐藏层
        
        layer_2_delta = layer_2_deltas[-position-1]   #当前时间点输出导数
        #通过后一个时间点的一隐藏层误差和当前时间点的输出层误差，计算当前时间点的隐藏层误差
        layer_1_delta = (fuature_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T))*sigmoid_output_to_derivative(layer_1)

        
        
        #等完成了所有反向传播误差计算，才会更新权重矩阵，先暂时把梯度矩阵存起来
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        fuature_layer_1_delta = layer_1_delta
        
    #完成所有反向传播之后，更新权重矩阵，并把矩阵梯度变量清0
    synapse_0 -= synapse_0_update*learning_rate
    synapse_1 -= synapse_1_update*learning_rate
    synapse_h -= synapse_h_update*learning_rate
    synapse_0_update = 0
    synapse_1_update = 0
    synapse_h_update = 0
    '''
    八 打印输出结果
    '''
    if i%800 == 0:
        print('总误差:',str(over_all_error))
        print('Pred:',str(d))
        print('True:',str(c))
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + '-' + str(b_int) + '=' + str(out))
        print('-------------------------------------------------')

总误差: [3.99972855]
Pred: [0 0 0 0 0 0 0 0]
True: [0 0 1 1 0 0 1 1]
60-9=0
-------------------------------------------------
总误差: [2.486562]
Pred: [0 0 0 0 0 0 0 0]
True: [0 0 0 1 0 0 0 1]
17-0=0
-------------------------------------------------
总误差: [3.51869416]
Pred: [0 0 1 0 0 1 1 0]
True: [0 0 0 1 1 1 1 0]
89-59=38
-------------------------------------------------
总误差: [0.18361106]
Pred: [0 0 0 1 1 0 0 0]
True: [0 0 0 1 1 0 0 0]
43-19=24
-------------------------------------------------
总误差: [0.1709148]
Pred: [0 0 0 0 0 0 1 0]
True: [0 0 0 0 0 0 1 0]
73-71=2
-------------------------------------------------
总误差: [0.13827615]
Pred: [0 0 1 1 1 1 0 0]
True: [0 0 1 1 1 1 0 0]
71-11=60
-------------------------------------------------
总误差: [0.08982648]
Pred: [1 0 0 0 0 0 0 0]
True: [1 0 0 0 0 0 0 0]
230-102=128
-------------------------------------------------
总误差: [0.17024705]
Pred: [0 1 1 1 0 0 0 1]
True: [0 1 1 1 0 0 0 1]
160-47=113
-------------------------------------------------
总误差